In [1]:
from robot_descriptions.z1_description import URDF_PATH
from robot_descriptions.z1_mj_description import MJCF_PATH


import mujoco
import pinocchio as pin
import numpy as np

np.random.seed(0)

In [2]:
pinmodel = pin.buildModelFromUrdf(URDF_PATH)
pindata = pinmodel.createData()

mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
mjdata = mujoco.MjData(mjmodel)

In [3]:
q = np.random.randn(mjmodel.nq)
v = np.random.randn(mjmodel.nv)
dv = np.random.randn(mjmodel.nv)

# tau = pin.rnea(pinmodel, pindata, q, v, dv)

mjdata.qpos = q.copy()
mjdata.qvel = v.copy()
mjdata.qacc = dv.copy()
# mjdata.ctrl = tau.copy()

In [4]:

body_idx = 4

In [5]:
pin.computeAllTerms(pinmodel, pindata, q, v)
pin.forwardKinematics(pinmodel, pindata, q, v, dv)
# pin.framesForwardKinematics(pinmodel, pindata, q)

In [6]:
def pin_body(idx):
    vel = pin.getVelocity(pinmodel, pindata, idx, pin.LOCAL)

    accel = pin.getAcceleration(pinmodel, pindata, idx, pin.LOCAL)

    rotation = pindata.oMi[idx].rotation

    return vel, accel, rotation



vel, acc, rotation = pin_body(body_idx)

vel, acc, rotation

(  v = -0.0386146  -0.213597 0.00697333
   w =   0.43721  0.156022 -0.843514,
   v = 0.0825157 -0.141572 0.0439487
   w =   0.48182  0.899213 -0.607455,
 array([[ 0.17051182, -0.9813841 ,  0.08837964],
        [-0.87129896, -0.19205531, -0.45161143],
        [ 0.46017805,  0.        , -0.88782665]]))

In [7]:
pin.getClassicalAcceleration(pinmodel, pindata, body_idx, pin.LOCAL)

  v = -0.0965684  -0.112049 -0.0434133
  w =   0.48182  0.899213 -0.607455

In [8]:
# mujoco.mj_step(mjmodel, mjdata)
# mujoco.mj_kinematics(mjmodel, mjdata)
mujoco.mj_inverse(mjmodel, mjdata)

In [9]:
# TODO: Read more about transformations here:
# https://mujoco.readthedocs.io/en/stable/programming/simulation.html#coordinate-frames-and-transformations
# velocity computation 2 - xbody
rotlin_vel = np.zeros(6)
mujoco.mj_objectVelocity(mjmodel, mjdata, 2, body_idx + 1, rotlin_vel, 1)

# acceleration computation
rotlin_acc = np.zeros(6)
mujoco.mj_rnePostConstraint(mjmodel, mjdata)
mujoco.mj_objectAcceleration(mjmodel, mjdata, 2, body_idx + 1, rotlin_acc, 1)

# rotation matrix
rotation = mjdata.xmat[body_idx + 1].reshape(3, 3).copy()
rotlin_vel, rotlin_acc, rotation
vel_lin_muj = rotlin_vel[3:]
vel_ang_muj = rotlin_vel[:3]
acc_lin_muj_g = rotlin_acc[3:]
acc_lin_muj_class = rotlin_acc[3:] + rotation.T @ mjmodel.opt.gravity
acc_ang_muj = rotlin_acc[:3] 
vel_lin_muj, vel_ang_muj, acc_lin_muj_class, acc_ang_muj, rotation

(array([-0.03861464, -0.21359706,  0.00697333]),
 array([ 0.43720984,  0.15602244, -0.84351382]),
 array([-0.09656842, -0.1120487 , -0.04341331]),
 array([ 0.48181994,  0.89921258, -0.60745503]),
 array([[ 1.70511825e-01, -9.81384103e-01,  8.83796400e-02],
        [-8.71298959e-01, -1.92055313e-01, -4.51611426e-01],
        [ 4.60178054e-01,  8.32667268e-17, -8.87826649e-01]]))

In [10]:
acc_muj_lin_spatial = acc_lin_muj_class - pin.skew(vel_ang_muj)@vel_lin_muj 
acc_muj_lin_spatial

array([ 0.08251565, -0.14157187,  0.04394868])

In [11]:
jacobian_pin = pin.getJointJacobian(pinmodel, 
                                pindata, 
                                body_idx, 
                                pin.LOCAL)
jacobian_pin[:3], jacobian_pin[3:]

(array([[-6.93889390e-18,  1.62745250e-01,  1.35459479e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-2.24818089e-01,  0.00000000e+00, -6.93889390e-18,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-3.46944695e-18, -1.68868885e-01,  1.80065903e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 array([[ 4.60178054e-01,  2.77555756e-17,  2.77555756e-17,
          2.77555756e-17,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-8.87826649e-01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]))

In [12]:
jacobian_muj_lin = np.zeros((3, mjmodel.nv))
jacobian_muj_rot = np.zeros((3, mjmodel.nv))

In [13]:
mujoco.mj_jacPointAxis(mjmodel, mjdata, jacobian_muj_lin, None, mjdata.xanchor[body_idx], mjdata.xaxis[body_idx], body_idx)
mujoco.mj_jacBody(mjmodel, mjdata, None, jacobian_muj_rot,  body_idx+1)
rotation = mjdata.xmat[body_idx+1].reshape(3, 3).copy()

In [14]:
rotation.T@jacobian_muj_lin, rotation.T@jacobian_muj_rot

(array([[ 7.57055331e-17,  1.62745250e-01,  1.35459479e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-2.86965954e-01,  3.14020881e-17,  3.53047835e-17,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 2.08373163e-17, -2.38868885e-01,  1.10065903e-01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
 array([[ 4.60178054e-01, -2.35287741e-16, -2.35287741e-16,
         -1.84197714e-16,  0.00000000e+00,  0.00000000e+00],
        [ 8.32667268e-17,  1.00000000e+00,  1.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-8.87826649e-01, -5.63852024e-17, -5.63852024e-17,
         -1.54953761e-16,  0.00000000e+00,  0.00000000e+00]]))